In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import os
import cv2
import torch
import pandas as pd
from PIL import Image
from pprint import pprint
import albumentations as albu
from albumentations.pytorch import ToTensorV2

from src.lightning_module import PosterModule

In [ ]:
DEVICE = 'cuda:0'

In [ ]:
# Загружаем модель

checkpoint_name = '../experiments/exp1/epoch_epoch=05-val_f1=0.132.ckpt'
module = PosterModule.load_from_checkpoint(checkpoint_name)

_ = module.eval()
_ = module.to(DEVICE)

In [ ]:
DATA_FOLDER = '/home/sergey/sdb3/datasets/Multi_Label_dataset/'
!ls {DATA_FOLDER}

In [ ]:
# Загружаем данные
df = pd.read_csv(os.path.join(DATA_FOLDER, 'df_test.csv'))
names = list(df.columns[1:])
names
df.head()

In [ ]:
# препроцессинг данных 
img_height = 224
img_width = 224
preprocess = albu.Compose(
        [
            albu.Resize(height=img_height, width=img_width),
            albu.Normalize(),
            ToTensorV2(),
        ]
)

In [ ]:
idx = 0
image_name = os.path.join(DATA_FOLDER, 'Images', f'{df["Id"][idx]}.jpg')
image = cv2.imread(image_name)[..., ::-1]
Image.fromarray(image)

In [ ]:
procecces_image = preprocess(image=image)['image']
with torch.no_grad():
    scores = torch.sigmoid(module(procecces_image[None].to(DEVICE)))[0].cpu().numpy()
    
pprint({n:s for s, n in zip(scores, names)})